# Chat API LLM Detector - Google Colab

Detect which chat API (ChatGPT, Claude, Grok, Gemini, or DeepSeek) generated a piece of text.

**Important:** Make sure to enable GPU runtime!
- Go to: Runtime → Change runtime type → Hardware accelerator → GPU (T4)

## Step 1: Install Dependencies

Installing compatible versions of all required packages...

**Note:** This project uses Llama 3, which requires Hugging Face authentication.

In [ ]:
# Install compatible versions to avoid dependency conflicts
!pip install -q transformers==4.46.3 peft==0.13.2 huggingface-hub accelerate
!pip install -q llm2vec==0.2.3

print("✓ Dependencies installed successfully!")

## Step 3: Download the Classifier

This will download ~16GB. It may take 5-10 minutes.

In [ ]:
from huggingface_hub import login

# Try to get token from Colab secrets
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("✓ Successfully authenticated with Hugging Face!")
except:
    # If secret fails, try manual input
    print("⚠️  Could not find HF_TOKEN in Colab secrets.")
    print("\nOption 1 - Add to Colab Secrets (recommended):")
    print("1. Request access: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct")
    print("2. Create token: https://huggingface.co/settings/tokens")
    print("3. Click the 🔑 icon on the left sidebar")
    print("4. Add a secret named 'HF_TOKEN' with your token")
    print("5. Enable notebook access toggle")
    print("6. Restart runtime and re-run this cell")
    print("\nOption 2 - Manual login (less secure):")
    from getpass import getpass
    token = getpass("Enter your HuggingFace token: ")
    login(token=token)
    print("✓ Authenticated!")

## Step 4: Load the Classifier

In [ ]:
!huggingface-cli download Yida/classifier_chat --local-dir ./classifier_chat

## Step 5: Define Prediction Function

In [ ]:
import torch
import numpy as np
from transformers import AutoConfig, AutoModel, AutoTokenizer
from peft import PeftModel
from llm2vec import LLM2Vec
import gc
import os

def load_classifier(checkpoint_path="./classifier_chat", num_labels=5):
    """Load the pre-trained LLM2Vec classifier with memory optimization."""
    print("Loading classifier with memory optimization...")
    
    # Clear cache first
    torch.cuda.empty_cache()
    gc.collect()
    
    base_model_name = "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp"
    
    config = AutoConfig.from_pretrained(
        base_model_name,
        trust_remote_code=True,
    )
    
    # Strategy: Try single GPU first, fall back to multi-GPU if OOM
    num_gpus = torch.cuda.device_count()
    print(f"Detected {num_gpus} GPU(s)")
    
    # Always use single GPU - multi-GPU causes device mismatch with residual connections
    # Even with 2 GPUs available, using just one with CPU offload is more reliable
    print("Using single GPU (GPU 0) with CPU offload for stability")
    device_map = {"": 0}  # Force everything to GPU 0
    max_memory = {0: "14GiB", "cpu": "30GiB"}
    
    # Load base model
    print("Loading base model...")
    model = AutoModel.from_pretrained(
        base_model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        device_map=device_map,
        max_memory=max_memory,
        offload_folder="./offload",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print("✓ Base model loaded")
    torch.cuda.empty_cache()
    
    # Load and merge first adapter
    print("Loading first adapter...")
    model = PeftModel.from_pretrained(
        model,
        base_model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    # Move to CPU for merging to avoid OOM
    print("✓ Moving to CPU for merging...")
    model = model.cpu()
    torch.cuda.empty_cache()
    gc.collect()
    
    print("✓ Merging first adapter...")
    model = model.merge_and_unload()
    
    # Move back to GPU 0 with CPU offload
    print("✓ Moving merged model back to GPU...")
    model = model.to(torch.bfloat16)
    
    # Re-dispatch to GPU 0 (with CPU offload for layers that don't fit)
    from accelerate import dispatch_model, infer_auto_device_map
    device_map_merged = infer_auto_device_map(
        model,
        max_memory=max_memory,
        no_split_module_classes=["LlamaDecoderLayer"]  # Keep decoder layers intact
    )
    model = dispatch_model(model, device_map=device_map_merged, offload_dir="./offload")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    # Load supervised adapter
    print("✓ Loading supervised adapter...")
    model = PeftModel.from_pretrained(
        model,
        f"{base_model_name}-supervised",
        is_trainable=True,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
    )
    
    torch.cuda.empty_cache()
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    model = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)
    
    hidden_size = list(model.modules())[-1].weight.shape[0]
    model.head = torch.nn.Linear(hidden_size, num_labels, dtype=torch.bfloat16)
    
    head_path = os.path.join(checkpoint_path, "head.pt")
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.head.load_state_dict(torch.load(head_path, map_location=device))
    model.head = model.head.to(device)
    
    model.eval()
    
    # Final cleanup
    torch.cuda.empty_cache()
    gc.collect()
    
    # Show memory usage across all GPUs
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            if i == 0 or allocated > 0:
                print(f"✓ GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    else:
        print("✓ Classifier loaded on CPU")
    
    return model

# Load the model
model = load_classifier()

## Step 6: Test with Sample Text

Replace the text below with any text you want to classify:

In [ ]:
def predict_text(model, text):
    """Predict which LLM generated the given text."""
    label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
    
    # Prepare text
    prepared_text = model.prepare_for_tokenization(text)
    inputs = model.tokenize([prepared_text])
    
    # IMPORTANT: For multi-GPU setups, always put inputs on cuda:0
    # The model will handle moving tensors between GPUs automatically
    if torch.cuda.is_available():
        target_device = torch.device("cuda:0")
    else:
        target_device = next(model.parameters()).device
    
    inputs = {k: v.to(target_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        embeddings = model.forward(inputs)
        
        # Move embeddings to same device as classification head
        if hasattr(model, 'head'):
            head_device = next(model.head.parameters()).device
            embeddings = embeddings.to(head_device)
        
        embeddings = embeddings.to(torch.bfloat16)
        logits = model.head(embeddings)
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    pred_label = torch.argmax(probabilities, dim=-1).item()
    # Convert to float32 before numpy (bfloat16 not supported by numpy)
    all_probs = probabilities[0].float().cpu().numpy()
    
    # Display results
    print("\n" + "="*60)
    print("PREDICTION RESULTS")
    print("="*60)
    print(f"\nMost likely source: {label_names[pred_label]}")
    print(f"Confidence: {all_probs[pred_label]*100:.2f}%")
    print("\nAll probabilities:")
    print("-"*60)
    
    sorted_indices = np.argsort(all_probs)[::-1]
    for idx in sorted_indices:
        bar_length = int(all_probs[idx] * 50)
        bar = "█" * bar_length
        print(f"{label_names[idx]:20s} {all_probs[idx]*100:6.2f}% {bar}")
    print("="*60)
    
    return label_names[pred_label], all_probs[pred_label]

## Step 7: Classify Your Own Text

Paste your text in the input box below:

In [ ]:
# Example text - replace with your own!
sample_text = """
Hello! I'd be happy to help you with that question. Let me break this down into a few key points:

1. First, it's important to understand the context
2. Second, we should consider the implications
3. Finally, let's look at practical applications

I hope this helps clarify things for you!
"""

predict_text(model, sample_text)

## Step 6: Classify Your Own Text

Paste your text in the input box below:

In [ ]:
# Interactive input
print("Paste your text below and press Enter:")
user_text = input()

if user_text.strip():
    predict_text(model, user_text)
else:
    print("No text provided!")

## Multiple Texts

You can test multiple texts at once:

In [ ]:
# Test multiple texts
texts_to_test = [
    "Sure, I can help with that!",
    "I'd be happy to assist you with this question.",
    "Let me break this down for you step by step.",
]

for i, text in enumerate(texts_to_test, 1):
    print(f"\n{'='*60}")
    print(f"TEXT #{i}: {text[:50]}...")
    predict_text(model, text)

In [ ]:
!pip install -q gradio pandas

import gradio as gr
import pandas as pd
import io
import sys
from contextlib import redirect_stdout, redirect_stderr

def predict_gradio(text):
    """Predict for Gradio interface with detailed logs."""
    if not text.strip():
        return "⚠️ Please enter some text to classify.", None, "⚠️ No text provided"
    
    # Capture all output (stdout and stderr)
    log_capture = io.StringIO()
    
    try:
        label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
        
        # Start logging
        log_capture.write("🔄 Starting prediction...\n\n")
        
        # Prepare text
        log_capture.write("📝 Preparing text for tokenization...\n")
        prepared_text = model.prepare_for_tokenization(text)
        log_capture.write(f"✓ Text prepared (length: {len(text)} chars)\n\n")
        
        # Tokenize
        log_capture.write("🔤 Tokenizing input...\n")
        inputs = model.tokenize([prepared_text])
        log_capture.write(f"✓ Tokenization complete\n\n")
        
        # Move to device
        log_capture.write("🖥️  Moving tensors to device...\n")
        if torch.cuda.is_available():
            target_device = torch.device("cuda:0")
            log_capture.write(f"✓ Using GPU: cuda:0\n\n")
        else:
            target_device = next(model.parameters()).device
            log_capture.write(f"✓ Using device: {target_device}\n\n")
        
        inputs = {k: v.to(target_device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        
        # Predict
        log_capture.write("🧠 Running model inference...\n")
        with torch.no_grad():
            embeddings = model.forward(inputs)
            log_capture.write(f"✓ Generated embeddings (shape: {embeddings.shape})\n")
            
            if hasattr(model, 'head'):
                head_device = next(model.head.parameters()).device
                embeddings = embeddings.to(head_device)
                log_capture.write(f"✓ Moved embeddings to classification head device\n")
            
            embeddings = embeddings.to(torch.bfloat16)
            logits = model.head(embeddings)
            probabilities = torch.nn.functional.softmax(logits, dim=-1)
            log_capture.write(f"✓ Computed probabilities\n\n")
        
        pred_label = torch.argmax(probabilities, dim=-1).item()
        all_probs = probabilities[0].float().cpu().numpy()
        
        # Log results
        log_capture.write("="*60 + "\n")
        log_capture.write("📊 PREDICTION RESULTS\n")
        log_capture.write("="*60 + "\n\n")
        log_capture.write(f"🎯 Most likely source: {label_names[pred_label]}\n")
        log_capture.write(f"💯 Confidence: {all_probs[pred_label]*100:.2f}%\n\n")
        log_capture.write("📈 All probabilities:\n")
        log_capture.write("-"*60 + "\n")
        
        sorted_indices = np.argsort(all_probs)[::-1]
        for idx in sorted_indices:
            bar_length = int(all_probs[idx] * 40)
            bar = "█" * bar_length
            log_capture.write(f"{label_names[idx]:20s} {all_probs[idx]*100:6.2f}% {bar}\n")
        
        log_capture.write("="*60 + "\n\n")
        log_capture.write("✅ Prediction complete!\n")
        
        # Create results for display
        result_text = f"## 🎯 Prediction: **{label_names[pred_label]}**\n\n"
        result_text += f"### Confidence: **{all_probs[pred_label]*100:.2f}%**\n\n"
        result_text += "---\n\n"
        result_text += "### All Probabilities:\n\n"
        
        for idx in sorted_indices:
            result_text += f"- **{label_names[idx]}**: {all_probs[idx]*100:.2f}%\n"
        
        # Create probability distribution for plot - using pandas DataFrame
        prob_data = pd.DataFrame({
            'Model': label_names,
            'Probability': [float(all_probs[i]) for i in range(len(label_names))]
        })
        # Sort by probability for better visualization
        prob_data = prob_data.sort_values('Probability', ascending=False)
        
        # Get all logs
        all_logs = log_capture.getvalue()
        
        return result_text, prob_data, all_logs
        
    except Exception as e:
        import traceback
        error_msg = f"❌ Error during prediction: {str(e)}\n\n"
        error_msg += f"Error type: {type(e).__name__}\n\n"
        error_msg += f"Traceback:\n{traceback.format_exc()}\n"
        log_capture.write(error_msg)
        return f"❌ Error: {str(e)}", pd.DataFrame(), log_capture.getvalue()


# Create Gradio interface with logs
with gr.Blocks(title="LLM Detector", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🔍 Chat API LLM Detector
        
        Detect which AI model (ChatGPT, Claude, Grok, Gemini, or DeepSeek) generated a piece of text.
        
        **Accuracy: 97.1%** | Based on [research paper](https://arxiv.org/abs/2502.12150)
        """
    )
    
    with gr.Row():
        with gr.Column(scale=2):
            text_input = gr.Textbox(
                label="Enter text to classify",
                placeholder="Paste the AI-generated text here...",
                lines=8,
            )
            
            submit_btn = gr.Button("🔍 Detect LLM", variant="primary", size="lg")
            
            gr.Examples(
                examples=[
                    ["Hello! I'd be happy to help you with that question. Let me break this down into a few key points."],
                    ["Sure, I can help with that! Here's what you need to know..."],
                    ["I'd be delighted to assist you with this matter."],
                ],
                inputs=text_input,
            )
    
    # Results section
    gr.Markdown("## 📊 Results")
    
    with gr.Row():
        with gr.Column(scale=1):
            result_output = gr.Markdown(value="", label="Prediction Results")
        
        with gr.Column(scale=1):
            plot_output = gr.BarPlot(
                value=pd.DataFrame({'Model': [], 'Probability': []}),
                x="Model",
                y="Probability", 
                title="Prediction Confidence",
                y_lim=[0, 1],
                height=300,
                width=400,
            )
    
    # Logs section
    gr.Markdown("## 📋 Processing Logs")
    logs_output = gr.Textbox(
        value="",
        label="Detailed logs",
        lines=15,
        max_lines=20,
        show_copy_button=True,
        interactive=False,
    )
    
    # Connect the button
    submit_btn.click(
        fn=predict_gradio,
        inputs=text_input,
        outputs=[result_output, plot_output, logs_output]
    )
    
    # Also trigger on enter
    text_input.submit(
        fn=predict_gradio,
        inputs=text_input,
        outputs=[result_output, plot_output, logs_output]
    )
    
    gr.Markdown(
        """
        ---
        **Note:** Works best on unedited AI-generated text. Accuracy decreases for short text (< 50 words).
        """
    )

# Launch the app
demo.launch(share=True, debug=True)

## Interactive Gradio UI

Launch an interactive web interface to classify text: